In [1]:
import numpy as np
import xarray as xr
import os
from tqdm.autonotebook import tqdm
import gcsfs
fs = gcsfs.GCSFileSystem() #list stores, stripp zarr from filename, load 

/tmp/ipykernel_11481/2254304399.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


Computes ASLC range and peak month based on averages by month derived from CMIP6 simlations

In [3]:
def open_zos_ibe_monmeans(zos_path,ibe_path,chunks):
    fns = ['gs://'+k for k in fs.ls(zos_path) if 'DS_Store' not in k]
    zos_monmeans = xr.open_mfdataset(fns,
                                 combine='nested',coords='minimal',compat='override',concat_dim = 'source_id',chunks=chunks,engine='zarr')
    
    fns = ['gs://'+k for k in fs.ls(ibe_path) if 'DS_Store' not in k]
    ibe_monmeans = xr.open_mfdataset(fns,
                                 combine='nested',coords='minimal',compat='override',concat_dim = 'source_id',chunks=chunks,engine='zarr')
    
    zos_ibe_monmeans = xr.merge((zos_monmeans,ibe_monmeans),join='inner')
    zos_ibe_monmeans['zos_ibe'] = zos_ibe_monmeans['zos'] + zos_ibe_monmeans['ibe']

    return zos_ibe_monmeans

def compute_range(ds):
    return ds.max(dim='month') - ds.min(dim='month')

def find_peak_months(ds):
    return 1+ds.argmax(dim='month',skipna=False).where(np.isfinite(ds.isel(month=0,drop=True))), 1+ds.argmin(dim='month',skipna=False).where(np.isfinite(ds.isel(month=0,drop=True)))

Configure the script:

In [4]:
ssps = ['ssp126']#['ssp126','ssp245','ssp370','ssp585'] #SSPs to process 

input_path_zos = 'gs://leap-persistent/timh37/CMIP6/zos_psmsl_month_means'
input_path_ibe = 'gs://leap-persistent/timh37/CMIP6/ibe_psmsl_month_means'

output_path = 'gs://leap-persistent/timh37/CMIP6/aslc_psmsl'
'''
input_path_zos = 'gs://leap-persistent/timh37/CMIP6/zos_1x1_month_means'
input_path_ibe = 'gs://leap-persistent/timh37/CMIP6/ibe_1x1_month_means'

output_path = 'gs://leap-persistent/timh37/CMIP6/aslc_1x1'
'''

In [5]:
for s,ssp in tqdm(enumerate(ssps)):
    if 'psmsl' in input_path_zos:
        monmeans = open_zos_ibe_monmeans(input_path_zos, #open gridded data
                                 input_path_ibe,{'tg':1000,'period':2,'member_id':200,'month':12}).sel(experiment_id=ssp)
    else:
        monmeans = open_zos_ibe_monmeans(input_path_zos, #open gridded data
                                 input_path_ibe,{'lat':500,'lon':500,'period':1,'member_id':50,'experiment_id':4,'month':12}).sel(experiment_id=ssp)

    #adjust string dtypes (otherwise zarr complains)
    monmeans['member_id'] = monmeans['member_id'].astype('str') 
    monmeans['source_id'] = monmeans['source_id'].astype('str')
    monmeans['experiment_id'] = monmeans['experiment_id'].astype('str')
    
    try:
        aslc_change_tgs['tg'] = aslc_change_tgs['tg'].astype('str')
    except:
        pass
    
    #compute aslc
    aslc_range =  100 * compute_range(monmeans) #cm
    aslc_amax,aslc_amin = find_peak_months(monmeans) #month 1-12

    #store
    aslc_range.to_zarr(os.path.join(output_path,'aslc_range_'+ssp+'.zarr'),mode='w',zarr_format=2) #circumvent issues with zarr3
    aslc_amax.to_zarr(os.path.join(output_path,'aslc_amax_'+ssp+'.zarr'),mode='w',zarr_format=2) #circumvent issues with zarr3

0it [00:00, ?it/s]